# 爬取南京房源，代码基于苏州房源py

In [1]:
import requests  # 用来访问网页的库，好比浏览器
from bs4 import BeautifulSoup
# 爬虫库，用来抓取网页中的信息，它有一个可爱的名字beautifulsoup,寓意一碗浓汤，我要从汤中捞出不同的美味
import time  # 时间模块，为了控制爬虫速度，防止由于速度过快，被 ban IP
import pandas as pd  # 用来操作csv的库，这次用来创建表格，存储数据
import csv
import codecs
import urllib
from fake_useragent import UserAgent  # 用于生成随机请求头
from multiprocessing import Pool

In [7]:
url_list = []  #建立一个空列表，用于存放二级网址


def get_url(start_num, end_num):  #建立获取网址的函数，参数为所需要提取的一级网址起始和终止页码
    url_page = [
        'https://nj.lianjia.com/ershoufang/pg{}l3l4l5l6a4a5p5p6p7/'.format(num)
        for num in range(start_num, end_num + 1)
    ]  #这是列表解析式，逻辑为根据起始和终止页码，拼接出指定数量的一级网址，放入列表待用
    n = 0  #设定一个计数变量
    for page in url_page:  #遍历每个一级网址
        n += 1  #计数加1
        print('正在抓取第{}页链接'.format(n))  #输出正在抓取的是第几个链接
        time.sleep(0.1)  #代码暂停运行1秒，为了控制爬虫速度，防止被 ban IP
        page_data = requests.get(page,
                                 headers={'User-Agent': UserAgent().random
                                          })  #用requests库的get方法访问一级网址，参数为链接和请求头
        soup = BeautifulSoup(
            page_data.text,
            'lxml')  #创建一个实例化BeautifulSoup类，类为一级网址的页面。意思就是根据配方熬一锅汤
        urls = soup.select(
            'body > div.content > div.leftContent > ul > li > div.info.clear > div.title > a'
        )
        #用selector选择器根据html标签,在页面中定位二级链接位置，最后返回的是一个列表(从一级网址爬取二级网址)
        for u in urls:  #遍历此列表，提取每一个位置
            url_list.append(u.get('href'))  #提取位置中链接的信息并存入开始创建的空列表中
    return url_list  #函数返回二级网址列表

In [19]:
def craw_inf(url_list):  #创建爬虫函数
    #file = codecs.open('C:/Users/123/Desktop/NanJing.csv','a+','utf-8') #创建并打开csv文件，参数为文件路径，打开方式ab+: 二进制附加读写方式打开，gbk编码
    #writer = csv.writer(file) #对csv进行写入
    #writer.writerow(['name','price','area_price','area','dist','lng','lat']) #写入表头
    n = 0  #设定一个计数变量
    for url in url_list:  #遍历每个二级网址
        n += 1  #计数加1
        print('正在抓取第{}个房源'.format(n))  #输出正在抓取的是第几个房源信息
        time.sleep(0.1)  #代码暂停运行1秒，为了控制爬虫速度，防止被 ban IP
        web = requests.get(url, headers={'User-Agent': UserAgent().random
                                         })  #用requests库的get方法访问二级网址，参数为链接和请求头
        soup = BeautifulSoup(web.text,
                             'lxml')  #创建一个实例化BeautifulSoup类，类为二级网址的页面
        try:
            namel = soup.select(
                'body > div.sellDetailHeader > div > div > div.title > h1'
            )  #selector选择器获取房源名称
            pricel = soup.select(
                'body > div.overview > div.content > div.price > span.total'
            )  #selector选择器获取总价
            area_pricel = soup.select(
                'body > div.overview > div.content > div.price > div.text > div.unitPrice > span'
            )  #selector选择器获取每平米价格
            areal = soup.select(
                'body > div.overview > div.content > div.houseInfo > div.area > div.mainInfo'
            )  #获取面积
            distl = soup.select(
                'body > div.overview > div.content > div.aroundInfo > div.areaName > span.info'
            )  #获取所在区域
            lng = soup.get_text().split("resblockPosition:'")[1].split(',')[
                0]  #根据页面结构获取经度
            lat = soup.get_text().split("resblockPosition:'")[1].split(
                ',')[1].split("'")[0]  #根据页面结构获取纬度

            for name, price, area_price, area, dist in zip(
                    namel, pricel, area_pricel, areal,
                    distl):  #遍历获取的每一个信息，提取具体的值
                output = '{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(
                    name.get_text(), price.get_text(), area_price.get_text(),
                    area.get_text(), dist.get_text(), lng, lat)  #把值以列表形式存入一个变量
                f = codecs.open('C:/Users/123/Desktop/NanJing.csv', 'a+',
                                'utf-8')  # 只能用xls格式
                f.write(output)
                f.close()
                print('Done')

        except IndexError:
            print('Error')

In [8]:
url_list = get_url(1, 100)  #这里设置了1到20页的范围

正在抓取第1页链接
正在抓取第2页链接
正在抓取第3页链接
正在抓取第4页链接
正在抓取第5页链接
正在抓取第6页链接
正在抓取第7页链接
正在抓取第8页链接
正在抓取第9页链接
正在抓取第10页链接
正在抓取第11页链接
正在抓取第12页链接
正在抓取第13页链接
正在抓取第14页链接
正在抓取第15页链接
正在抓取第16页链接
正在抓取第17页链接
正在抓取第18页链接
正在抓取第19页链接
正在抓取第20页链接
正在抓取第21页链接
正在抓取第22页链接
正在抓取第23页链接
正在抓取第24页链接
正在抓取第25页链接
正在抓取第26页链接
正在抓取第27页链接
正在抓取第28页链接
正在抓取第29页链接
正在抓取第30页链接
正在抓取第31页链接
正在抓取第32页链接
正在抓取第33页链接
正在抓取第34页链接
正在抓取第35页链接
正在抓取第36页链接
正在抓取第37页链接
正在抓取第38页链接
正在抓取第39页链接
正在抓取第40页链接
正在抓取第41页链接
正在抓取第42页链接
正在抓取第43页链接
正在抓取第44页链接
正在抓取第45页链接
正在抓取第46页链接
正在抓取第47页链接
正在抓取第48页链接
正在抓取第49页链接
正在抓取第50页链接
正在抓取第51页链接
正在抓取第52页链接
正在抓取第53页链接
正在抓取第54页链接
正在抓取第55页链接
正在抓取第56页链接
正在抓取第57页链接
正在抓取第58页链接
正在抓取第59页链接
正在抓取第60页链接
正在抓取第61页链接
正在抓取第62页链接
正在抓取第63页链接
正在抓取第64页链接
正在抓取第65页链接
正在抓取第66页链接
正在抓取第67页链接
正在抓取第68页链接
正在抓取第69页链接
正在抓取第70页链接
正在抓取第71页链接
正在抓取第72页链接
正在抓取第73页链接
正在抓取第74页链接
正在抓取第75页链接
正在抓取第76页链接
正在抓取第77页链接
正在抓取第78页链接
正在抓取第79页链接
正在抓取第80页链接
正在抓取第81页链接
正在抓取第82页链接
正在抓取第83页链接
正在抓取第84页链接
正在抓取第85页链接
正在抓取第86页链接
正在抓取第87页链接
正在抓取第88页链接
正在抓取第89页链接
正在抓取第90页链接
正在抓取第91页链接
正在抓取第92页

In [ ]:
craw_inf(url_list)

正在抓取第1个房源
Done
正在抓取第2个房源
Done
正在抓取第3个房源
Done
正在抓取第4个房源
Done
正在抓取第5个房源
Done
正在抓取第6个房源
Done
正在抓取第7个房源
Done
正在抓取第8个房源
Done
正在抓取第9个房源
Done
正在抓取第10个房源
Done
正在抓取第11个房源
Done
正在抓取第12个房源
Done
正在抓取第13个房源
Done
正在抓取第14个房源
Done
正在抓取第15个房源
Done
正在抓取第16个房源
Done
正在抓取第17个房源
Done
正在抓取第18个房源
Done
正在抓取第19个房源
Done
正在抓取第20个房源
Done
正在抓取第21个房源
Done
正在抓取第22个房源
Done
正在抓取第23个房源
Done
正在抓取第24个房源
Done
正在抓取第25个房源
Done
正在抓取第26个房源
Done
正在抓取第27个房源
Done
正在抓取第28个房源
Done
正在抓取第29个房源
Done
正在抓取第30个房源
Done
正在抓取第31个房源
Done
正在抓取第32个房源
Done
正在抓取第33个房源
Done
正在抓取第34个房源
Done
正在抓取第35个房源
Done
正在抓取第36个房源
Done
正在抓取第37个房源
Done
正在抓取第38个房源
Done
正在抓取第39个房源
Done
正在抓取第40个房源
Done
正在抓取第41个房源
Done
正在抓取第42个房源
Done
正在抓取第43个房源
Done
正在抓取第44个房源
Done
正在抓取第45个房源
Done
正在抓取第46个房源
Done
正在抓取第47个房源
Done
正在抓取第48个房源
Done
正在抓取第49个房源
Done
正在抓取第50个房源
Done
正在抓取第51个房源
Done
正在抓取第52个房源
Done
正在抓取第53个房源
Done
正在抓取第54个房源
Done
正在抓取第55个房源
Done
正在抓取第56个房源
Done
正在抓取第57个房源
Done
正在抓取第58个房源
Done
正在抓取第59个房源
Done
正在抓取第60个房源
Done
正在抓取第61个房源
Done
正在抓取第62个房源
Done
正在抓取第63个房源
Done
正

Done
正在抓取第490个房源
Done
正在抓取第491个房源
Done
正在抓取第492个房源
Done
正在抓取第493个房源
Done
正在抓取第494个房源
Done
正在抓取第495个房源
Done
正在抓取第496个房源
Done
正在抓取第497个房源
Done
正在抓取第498个房源
Done
正在抓取第499个房源
Done
正在抓取第500个房源
Done
正在抓取第501个房源
Done
正在抓取第502个房源
Done
正在抓取第503个房源
Done
正在抓取第504个房源
Done
正在抓取第505个房源
Done
正在抓取第506个房源
Done
正在抓取第507个房源
Done
正在抓取第508个房源
Done
正在抓取第509个房源
Done
正在抓取第510个房源
Done
正在抓取第511个房源
Done
正在抓取第512个房源
Done
正在抓取第513个房源
Done
正在抓取第514个房源
Done
正在抓取第515个房源
Done
正在抓取第516个房源
Done
正在抓取第517个房源
Done
正在抓取第518个房源
Done
正在抓取第519个房源
Done
正在抓取第520个房源
Done
正在抓取第521个房源
Done
正在抓取第522个房源
Done
正在抓取第523个房源
Done
正在抓取第524个房源
Done
正在抓取第525个房源
Done
正在抓取第526个房源
Done
正在抓取第527个房源
Done
正在抓取第528个房源
Done
正在抓取第529个房源
Done
正在抓取第530个房源
Done
正在抓取第531个房源
Done
正在抓取第532个房源
Done
正在抓取第533个房源
Done
正在抓取第534个房源
Done
正在抓取第535个房源
Done
正在抓取第536个房源
Done
正在抓取第537个房源
Done
正在抓取第538个房源
